In [1]:
import os, sys

colab = False
# #### COLAB ONLY CELL
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # drive.mount('/content/gdrive',force_remount=True)
    os.chdir("/content/gdrive/MyDrive/COMEXT")
    !pip install -q -r world-trade-thesis/requirements.txt
    # !pwd
    sys.path.append('/content/gdrive/MyDrive/COMEXT/world-trade-thesis/src')
else:
    os.chdir("H:/My Drive/COMEXT")
    print(os.getcwd())
    sys.path.append('H:/My Drive/COMEXT/world-trade-thesis/src')

# import random, math, os, pickle, sys, powerlaw, hashlib #,eurostat
# import pandas as pd
# import numpy as np
# import networkx as nx
# from tqdm.notebook import tqdm, trange
# from networkx.readwrite import json_graph
# from networkx.classes.function import degree
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# from scipy.optimize import curve_fit
# from sklearn.preprocessing import MinMaxScaler

import graph_analysis as g
if colab:
    import tqdm.notebook as tqdm, trange
# exec(open("world-trade-thesis/src/graph_analysis.py","r").read())

# df_pop, eu_iso = g.load_population_df()

H:\My Drive\COMEXT
Functions loaded!


In [3]:
import pickle
import pandas as pd
# from tqdm import tqdm
import networkx as nx
# import graph_analysis as g
from tqdm.notebook import tqdm, trange
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Running on whole time and merging to big df...")
metrics_full = {}
df_pop, eu_iso = g.load_population_df()

big_df = pd.DataFrame()
issues = []
for y in tqdm([y for y in range(2001,2021)]):
    df = g.load_filtered_data("complete", save=False, force_reload=True, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
                                years=[y], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")
    n4g = g.get_world_countries(df_pop, eu_iso, year=str(y))
    # for prod in tqdm(df.PRODUCT_CPA2_1.unique(),leave=False):
    for prod in ["01","10","06","35"]:
        df4g = df[df.PRODUCT_CPA2_1 == prod]
        t4g = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by='population', pop_df=df_pop)
        if t4g.shape[0] < 10:
            issues.append((y,prod,t4g.shape[0]))
        coord, metrics, G = g.makeGraph(t4g, tab_nodes=n4g, weight_flag=True, criterio="VALUE_IN_EUROS", compute_layout=False, colab=colab)
        nx.write_gexf(G,f"data-samples/graphs/tab_nodes/complete_y{y}_p{prod}.gexf")
        x = metrics.reset_index().to_dict()
        x["year"] = str(y)
        x["month"] = "-"
        x["prod"] = prod
        big_df = pd.concat([big_df,pd.DataFrame(x)],axis=0)
        metrics_full[f"{y}_cpa_{prod}"] = metrics
    #     break
    # break

version = 4

with open(f"./data-samples/metrics/issues_complete_{version}.txt","w") as f:
    f.write(str(issues))
pickle.dump(metrics_full,open(f"./data-samples/metrics/metrics_complete_V_{version}_t4g.pickle","wb"))
big_df.to_parquet(f"./data-samples/metrics/metrics_complete_V_{version}_t4g.parquet")


Running on whole time and merging to big df...


In [ ]:
g.load_filtered_data("complete", save=False, force_reload=True, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, 
    group_by_prod_code=True, n_digits=2, years=[y], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", 
    trade_type="all")

In [4]:
from tqdm.notebook import tqdm, trange

In [8]:
df_pop, eu_iso = g.load_population_df()
thresholds = {str(y):{} for y in range(2001,2021)}
for y in trange(2001,2021):
    df_full = g.load_filtered_data("complete", save=False, force_reload=True, verbose=False, sorted=False, columns=g.full_columns, 
                    types=g.full_types, group_by_prod_code=True, n_digits=2, years=[y], months=None, product="all", flow="all", 
                    declarant_iso="all", partner_iso="all", trade_type="all")
    for prod in tqdm(df_full.PRODUCT_CPA2_1.unique(), leave=False):
        df4g = df_full[df_full.PRODUCT_CPA2_1 == prod]
        df_full_scaled = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by="population",pop_df=df_pop)
        df_thresh = df_full_scaled.sort_values("VALUE_IN_EUROS_SCALED",ascending=False).reset_index(drop=True)
        df_thresh = df_thresh[df_thresh.VALUE_IN_EUROS_RESCALED.cumsum() < 0.9999]
        thresholds[str(y)][prod] = df_thresh.iloc[-1,-4]

In [10]:
import pandas as pd
pd.DataFrame(thresholds).to_csv("data-samples/manual/complete_thresholds2.csv")